# Complex body TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a model with a complex geometry.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [22]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [23]:
model = 'complex_data/complex_model.pickle'
data = 'complex_data/regular_grid_50x50_100km/tfa_data.pickle'

In [24]:
with open(model) as w:
        complex_model = pickle.load(w)

### Observed data

In [25]:
with open(data) as w:
        complex_data = pickle.load(w)

In [26]:
# observed data and observation points
dobs = complex_data['tfa_obs']
xp = complex_data['grid'][0]
yp = complex_data['grid'][1]
zp = complex_data['grid'][2]
N = complex_data['grid'][3]

### Parameters of the initial model

In [27]:
M = 30 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = complex_model['inc']
decs = complex_model['dec']

# depth to the top, thickness and radius
z0 = complex_model['z0']
dz = 300.
r = 1000.

# total magnetization
props = {'magnetization': utils.ang2vec(
        complex_model['intensity'], incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.array([1000., 1000.])))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = complex_data['main_field']

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Limits

In [28]:
# limits for parameters in meters
rmin = 100.
rmax = 3000.
x0min = -3000.
x0max = 3000.
y0min = -3000.
y0max = 3000.
dzmin = 100.
dzmax = 500.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [29]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [30]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = complex_model['param_vec'][:M+2]

### Folder to save the results

In [31]:
if complex_data['grid'][4] == 'flightlines':
    if incs == complex_model['inc'] and decs == complex_model['dec']:
        mypath = 'tfa_inversion/true_direction_flightlines/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_flightlines/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == complex_model['inc'] and decs == complex_model['dec']:
        mypath = 'tfa_inversion/true_direction_regular/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_regular/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [32]:
# output of inversion
complex_inversion = dict()

### Regularization parameters

In [33]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.001  # adjacent radial distances within each prism
a2 = 0.01   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.01     # vertically adjacent origins
a6 = 0.001   # zero order Tikhonov on adjacent radial distances
a7 = 0.00001     # zero order Tikhonov on thickness of each prism

In [34]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [35]:
itmax = 30
itmax_marq = 10
tol = 1.0e-5     # stop criterion

In [36]:
complex_inversion['x'] = xp
complex_inversion['y'] = yp
complex_inversion['z'] = zp
complex_inversion['observed_data'] = dobs

In [37]:
complex_inversion['inc_dec'] = [incs, decs]
complex_inversion['z0'] = z0
complex_inversion['initial_dz'] = dz
complex_inversion['intial_r'] = r
complex_inversion['initial_estimate'] = model0
complex_inversion['initial_data'] = d0
complex_inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
complex_inversion['tol'] = tol
complex_inversion['main_field'] = [complex_data['main_field'][0], complex_data['main_field'][1]]

### Inversion

In [38]:
d_fit, m_est, model_est, phi_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+03   misfit: 1.81361e+05
it:  1   it_marq:  0   lambda: 1e+02   misfit: 1.47053e+05
it:  2   it_marq:  0   lambda: 1e+01   misfit: 8.19744e+04
it:  3   it_marq:  0   lambda: 1e+00   misfit: 6.47988e+04
it:  4   it_marq:  0   lambda: 1e-01   misfit: 5.54295e+04
it:  5   it_marq:  0   lambda: 1e-02   misfit: 5.72813e+04
it:  5   it_marq:  1   lambda: 1e-01   misfit: 7.51382e+04
it:  5   it_marq:  2   lambda: 1e+00   misfit: 1.39817e+05
it:  5   it_marq:  3   lambda: 1e+01   misfit: 1.78756e+05
it:  5   it_marq:  4   lambda: 1e+02   misfit: 1.33150e+05
it:  5   it_marq:  5   lambda: 1e+03   misfit: 1.26715e+05
it:  5   it_marq:  6   lambda: 1e+04   misfit: 5.78021e+04
it:  5   it_marq:  7   lambda: 1e+05   misfit: 6.10223e+04
it:  5   it_marq:  8   lambda: 1e+06   misfit: 6.27867e+04
it:  5   it_marq:  9   lambda: 1e+07   misfit: 6.29258e+04
it:  6   it_marq:  0   lambda: 1e+08   misfit: 6.30495e+04
it:  6   it_marq:  1   lambda: 1e+09   misfit: 6.30495e+

# Results

In [39]:
complex_inversion['data_fit'] = d_fit
complex_inversion['estimate'] = m_est
complex_inversion['prisms'] = model_est
complex_inversion['objective'] = phi_list
complex_inversion['residual'] = dobs - d_fit

In [40]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(complex_inversion, f)